# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

## Подготовка

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier
from nltk.corpus import stopwords
from pymystem3 import Mystem

import pandas as pd
import numpy as np
import nltk
import re

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv')
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [4]:
nltk.download('stopwords', quiet=True)
stop_words = set(stopwords.words('english'))

In [5]:
def clear_text(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text)  
    return " ".join(text.split()).lower()

In [6]:
df.text = df.text.apply(clear_text)

In [7]:
X = df.drop('toxic', axis=1)
y = df['toxic']

X_trains, X_test, y_trains, y_test = train_test_split(X, y, test_size=.2, random_state=123)
X_train, X_valid, y_train, y_valid = train_test_split(X_trains, y_trains, test_size=.2, random_state=123)

In [8]:
count_tf_idf = TfidfVectorizer(stop_words=stop_words)
tf_train = count_tf_idf.fit_transform(X_train.text)
tf_valid = count_tf_idf.transform(X_valid.text)
tf_test = count_tf_idf.transform(X_test.text)

### Шаги:
    1.Подключены нужные библиотеки и выведена первичная информация
    2.В переменную stop_words сохранены стоп слова
    3.Написана функция clear_text, которая очищает текст от всех символов кроме a-z и A-Z
    4.К столбцу text применена функция clear_text
    5.Датафрейм разбит на тренировочную, валидационную и тестовую выборки
    6.Рассчитана TF-IDF для тренировочной, валидационной и тестовой выборки

## Обучение

In [9]:
LogisticRegression_model = LogisticRegression()


parametrs = {
    'C': [0.01, 0.1, 1, 10, 20],
}

grid = GridSearchCV(LogisticRegression_model, parametrs, cv=5)

grid.fit(tf_train, y_train)

grid.best_params_

{'C': 10}

In [10]:
LogisticRegression_model = LogisticRegression(C=10).fit(tf_train, y_train)
LogisticRegression_prediction = LogisticRegression_model.predict(tf_valid)
f1_score(y_valid, LogisticRegression_prediction)

0.7706537733865881

In [11]:
LGBMClassifier_model = LGBMClassifier()


parametrs = {
    'n_estimators': range(1, 100, 20),
    'max_depth': range(1, 10, 5)
}

grid = GridSearchCV(LGBMClassifier_model, parametrs, cv=5)

grid.fit(tf_train, y_train)

grid.best_params_

{'max_depth': 6, 'n_estimators': 81}

In [12]:
LGBMClassifier_model = LGBMClassifier(max_depth=6, n_estimators=81).fit(tf_train, y_train)
LGBMClassifier_prediction = LGBMClassifier_model.predict(tf_valid)
f1_score(y_valid, LGBMClassifier_prediction)

0.6254653760238272

In [13]:
prediction = LogisticRegression_model.predict(tf_test)
f1_score(y_test, prediction)

0.772337383845604

### Шаги:
    1.С помощью GridSearchCV определены наилучшие параметры для модели LogisticRegression
    2.Обучена модель LogisticRegression с параметром C=10
    3.Обучена модель LGBMClassifier с тремя параметрами max_depth=6, n_estimators=81
    4.Обучена модель LogisticRegression на тестовой выборки

## Выводы

    Лучшая модель LogisticRegression. Более простая модель отработала лучше и дала нужный нам результат f1 на тестовой выборке больше 75.